# 최소 비용 최대 유량 (Minimum Cost Maximum Flow)

`-` 최대 유량을 흘리되 비용을 최소로 하자

## 열혈강호 5

- 문제 출처: [백준 11408번](https://www.acmicpc.net/problem/11408)

`-` 간선에 비용이 추가됐다

`-` 비용이 없고 유량과 용량만 있는 그래프에서 최대 유량을 찾을 땐 BFS를 통해 증가 경로를 찾았다

`-` 여기서는 최소 비용인 증가 경로를 찾기 위해 최단 거리 탐색 알고리즘을 사용할 것이다

`-` 단, 역방향 간선의 경우 유량이 음수일 수 있으므로 음수 간선이 존재해도 잘 동작하는 Bellman-Ford 알고리즘을 사용하자

`-` Bellman-Ford 알고리즘을 개량한 SPFA(Shortest Path Faster Algorithm)를 사용할 것이다

`-` Bellman-Ford 알고리즘에선 매 반복문마다 모든 간선을 순회하며 거리 갱신을 시도했지만 SPFA에선 거리가 갱신된 노드와 연결된 간선만 고려할 것이다

`-` 이러한 방식의 알고리즘을 SSP(Successive Shortest Path) 알고리즘이라 한다

`-` SPFA의 시간 복잡도는 최악의 경우 $O(VE)$이므로 SSP 알고리즘의 시간 복잡도는 $O(VEf)$이다 (단, $f$는 최대 유량)

`-` 매 증가 경로마다 최소 비용으로 구했기에 최대 유량을 흘릴 때의 비용 또한 최소 비용이다 (그렇지 않다면 모순이다)

`-` 참고: https://cp-algorithms.com/graph/min_cost_flow.html

`-` 참고: https://codeforces.com/blog/entry/105330

`-` 이 문제에선 최대 유량이 강호네 회사에서 할 수 있는 일의 개수의 최댓값이고 최소 비용이 일을 최대로 할 때 강호가 지급해야 하는 월급의 최솟값이다

In [28]:
from collections import defaultdict, deque


def ssp(graph, weight, source, sink, flow, capacity, num_nodes):
    max_flow = 0
    min_cost = 0
    exists_augmenting_path = True
    while exists_augmenting_path:
        predecessors, distances = spfa(graph, weight, source, flow, capacity, num_nodes)
        exists_augmenting_path = predecessors[sink] != NONE
        if not exists_augmenting_path:
            break
        augmenting_path = track_augmenting_path(predecessors, sink)
        bottleneck = compute_bottleneck(augmenting_path, flow, capacity)
        update_flow(augmenting_path, flow, bottleneck)
        cost = distances[sink]
        max_flow += bottleneck
        min_cost += cost
    return max_flow, min_cost


def spfa(graph, weight, source, flow, capacity, num_nodes):
    queue = deque([source])
    distances = [INF] * num_nodes
    distances[source] = 0
    predecessors = [NONE] * num_nodes
    in_queue = [False] * num_nodes
    in_queue[source] = True
    while queue:
        u = queue.popleft()
        dist_u = distances[u]
        in_queue[u] = False
        for v in graph[u]:
            w = weight[u][v]
            residual_capacity = capacity[u][v] - flow[u][v]
            dist_v = distances[v]
            if residual_capacity <= 0 or dist_v <= dist_u + w:
                continue
            distances[v] = dist_u + w
            predecessors[v] = u
            if in_queue[v]:
                continue
            in_queue[v] = True
            queue.append(v)
    return predecessors, distances


def track_augmenting_path(predecessors, sink):
    augmenting_path = []
    node = sink
    while node != NONE:
        augmenting_path.append(node)
        node = predecessors[node]
    augmenting_path.reverse()
    return augmenting_path


def compute_bottleneck(augmenting_path, flow, capacity):
    bottleneck = INF
    for i in range(len(augmenting_path) - 1):
        u, v = augmenting_path[i], augmenting_path[i + 1]
        bottleneck = min(capacity[u][v] - flow[u][v], bottleneck)
    return bottleneck


def update_flow(augmenting_path, flow, bottleneck):
    for i in range(len(augmenting_path) - 1):
        u, v = augmenting_path[i], augmenting_path[i + 1]
        flow[u][v] += bottleneck
        flow[v][u] -= bottleneck


def solution():
    global INF, NONE
    N, M = map(int, input().split())
    INF = float("inf")
    NONE = -1
    source = 0
    sink = N + M + 1
    num_nodes = N + M + 2
    flow = [[0] * num_nodes for _ in range(num_nodes)]
    capacity = [[1] * (num_nodes) for _ in range(num_nodes)]
    graph = defaultdict(list)
    graph[source] = list(range(1, N + 1))
    weight = [[0] * num_nodes for _ in range(num_nodes)]
    for work in range(N + 1, N + M + 1):
        graph[work].append(sink)
    for employee in range(1, N + 1):
        data = list(map(int, input().split()))
        n = data[0]
        for i in range(n):
            work = data[2 * i + 1] + N
            salary = data[2 * (i + 1)]
            graph[employee].append(work)
            graph[work].append(employee)
            capacity[work][employee] = 0
            weight[employee][work] = salary
            weight[work][employee] = -salary
    max_flow, min_cost = ssp(graph, weight, source, sink, flow, capacity, num_nodes)
    print(max_flow)
    print(min_cost)


solution()

# input
# 5 5
# 2 1 3 2 2
# 1 1 5
# 2 2 1 3 7
# 3 3 9 4 9 5 9
# 1 1 0

 5 5
 2 1 3 2 2
 1 1 5
 2 2 1 3 7
 3 3 9 4 9 5 9
 1 1 0


4
18


`-` spfa 구현할 때 큐에 넣으면 빼먹지 말고 큐에 들어있다고 표시해줘야 한다

`-` 처음엔 weight 행렬을 defaultdict으로 구현했었다

`-` 기본 값은 $0$으로 주었는데 자료 구조 자체가 느린 건지 아니면 부작용이 있었던 것인지 $2\%$도 못 가고 시간 초과가 계속 발생했다

`-` 이중 리스트로 구현하니까 바로 맞혔다

## 열혈강호 6

- 문제 출처: [백준 11409번](https://www.acmicpc.net/problem/11409)

`-` 일을 최대한 많이 시키되 강호가 지불해야 하는 월급도 최대로 해야 한다

`-` 최대 비용 최대 유량 문제(?)인데 강호가 지불하는 월급을 음수로 설정하면 그만이다

`-` 파이썬에서 힙을 사용할 때 최소 힙만 지원해서 최대 힙으로 사용하고 싶을 땐 원소에 $-1$을 곱해서 사용했었다

`-` 이것처럼 정방향 간선의 비용에 $-1$을 곱해주면 된다 (역방향 간선의 비용은 정방향 간선에 $-1$을 곱하면 됨)

`-` 최대 유량과 그 때의 최소 비용을 계산했으면 최소 비용에 다시 $-1$을 곱해주면 강호가 지불하는 월급의 최댓값을 알 수 있다

`-` 참고로 유량을 흘리다보면 음수 사이클이 생기지 않을까 걱정할 수 있다

`-` 만약 그래프에 없던 음수 사이클이 새로 생겼다면 유량을 흘려주고 역방향 간선이 활성화 되면서 생겼다는 의미이다

`-` 최소 비용 경로에 존재하는 간선에 대해 역방향 간선이 생기게 된다

`-` 해당 역방향 간선을 따라 음수 사이클이 생기는 것을 고려할 수 있는데 최소 비용인지라 사이클 내의 정방향 간선 쪽 비용과 합하면 $0$보다 클 수 밖에 없다

`-` 따라서 처음 그래프에 음수 사이클이 없다면 새롭게 음수 사이클이 생기지 않으니 걱정하지 않아도 된다

In [33]:
from collections import defaultdict, deque


def ssp(graph, weight, source, sink, flow, capacity, num_nodes):
    max_flow = 0
    min_cost = 0
    exists_augmenting_path = True
    while exists_augmenting_path:
        predecessors, distances = spfa(graph, weight, source, flow, capacity, num_nodes)
        exists_augmenting_path = predecessors[sink] != NONE
        if not exists_augmenting_path:
            break
        augmenting_path = track_augmenting_path(predecessors, sink)
        bottleneck = compute_bottleneck(augmenting_path, flow, capacity)
        update_flow(augmenting_path, flow, bottleneck)
        cost = distances[sink]
        max_flow += bottleneck
        min_cost += cost
    return max_flow, min_cost


def spfa(graph, weight, source, flow, capacity, num_nodes):
    queue = deque([source])
    distances = [INF] * num_nodes
    distances[source] = 0
    predecessors = [NONE] * num_nodes
    in_queue = [False] * num_nodes
    in_queue[source] = True
    while queue:
        u = queue.popleft()
        in_queue[u] = False
        dist_u = distances[u]
        for v in graph[u]:
            w = weight[u][v]
            dist_v = distances[v]
            residual_capacity = capacity[u][v] - flow[u][v]
            if residual_capacity <= 0 or dist_v <= dist_u + w:
                continue
            distances[v] = dist_u + w
            predecessors[v] = u
            if in_queue[v]:
                continue
            in_queue[v] = True
            queue.append(v)
    return predecessors, distances


def track_augmenting_path(predecessors, sink):
    augmenting_path = []
    node = sink
    while node != NONE:
        augmenting_path.append(node)
        node = predecessors[node]
    augmenting_path.reverse()
    return augmenting_path


def compute_bottleneck(augmenting_path, flow, capacity):
    bottleneck = INF
    for i in range(len(augmenting_path) - 1):
        u, v = augmenting_path[i], augmenting_path[i + 1]
        bottleneck = min(capacity[u][v] - flow[u][v], bottleneck)
    return bottleneck


def update_flow(augmenting_path, flow, bottleneck):
    for i in range(len(augmenting_path) - 1):
        u, v = augmenting_path[i], augmenting_path[i + 1]
        flow[u][v] += bottleneck
        flow[v][u] -= bottleneck


def solution():
    global INF, NONE
    N, M = map(int, input().split())
    INF = float("inf")
    NONE = -1
    source = 0
    sink = N + M + 1
    num_nodes = N + M + 2
    flow = [[0] * num_nodes for _ in range(num_nodes)]
    capacity = [[1] * (num_nodes) for _ in range(num_nodes)]
    graph = defaultdict(list)
    graph[source] = list(range(1, N + 1))
    weight = [[0] * num_nodes for _ in range(num_nodes)]
    for work in range(N + 1, N + M + 1):
        graph[work].append(sink)
    for employee in range(1, N + 1):
        data = list(map(int, input().split()))
        n = data[0]
        for i in range(n):
            work = data[2 * i + 1] + N
            salary = data[2 * (i + 1)]
            graph[employee].append(work)
            graph[work].append(employee)
            capacity[work][employee] = 0
            weight[employee][work] = -salary
            weight[work][employee] = salary
    max_flow, min_cost = ssp(graph, weight, source, sink, flow, capacity, num_nodes)
    max_cost = -min_cost
    print(max_flow)
    print(max_cost)


solution()

# input
# 5 5
# 2 1 3 2 2
# 1 1 5
# 2 2 1 3 7
# 3 3 9 4 9 5 9
# 1 1 0

 5 5
 2 1 3 2 2
 1 1 5
 2 2 1 3 7
 3 3 9 4 9 5 9
 1 1 0


4
23
